# Rebonato-Denev Bayesian Networks for Black Swan Events

## Interactive Tutorial

This notebook demonstrates the Bayesian network methodology from **"Portfolio Management under Stress"** by Riccardo Rebonato and Alexander Denev.

### Key Concept

Instead of relying solely on historical correlations (which break down during crises), we model the **causal structure** of how extreme events propagate through financial markets.

In [ ]:
# Import the implementation
import sys
sys.path.append('.')

from rebonato_denev_eurozone_crisis import (
    BayesianNetwork,
    build_eurozone_crisis_network,
    analyze_stress_scenarios,
    portfolio_implications
)

import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

## Step 1: Build the Bayesian Network

The network models the Eurozone crisis as a "black swan" event with clear causal structure:

```
Political_Instability ──┐
                        ├──> Eurozone_Breakup ──> Credit_Spreads ──> Corporate_Bonds
Economic_Weakness ─────┘                     │                   │
                                             │                   └──> Equities
                                             │                         ↑
                                             └──> Flight_to_Quality ───┤
                                                         │              │
                                                         └──> Government_Bonds
```

In [ ]:
# Build the network
bn = build_eurozone_crisis_network()

# Visualize
bn.visualize()

## Step 2: Define Scenarios

We'll analyze four different scenarios:

1. **Normal Times**: Low political instability, low economic weakness
2. **Black Swan**: High political instability + high economic weakness
3. **Eurozone Breakup**: Given that the breakup occurs
4. **Market Stress**: Observing widening spreads and flight to quality

In [ ]:
# Analyze all scenarios
results = analyze_stress_scenarios(bn)

## Step 3: Custom Scenario Analysis

You can define your own scenarios by setting evidence for specific variables.

In [ ]:
# Define a custom scenario
custom_evidence = {
    'Political_Instability': 'High',
    'Economic_Weakness': 'Low'  # Economic ok, but politics unstable
}

# Calculate probabilities
custom_results = bn.get_probability(custom_evidence)

print("Custom Scenario: High Political Instability, Low Economic Weakness")
print("=" * 70)
for var, probs in custom_results.items():
    print(f"\n{var}:")
    for state, prob in probs.items():
        print(f"  {state}: {prob:.1%}")

## Step 4: Monte Carlo Simulation

Run thousands of simulations to understand the distribution of outcomes under different scenarios.

In [ ]:
# Run Monte Carlo simulation under black swan scenario
black_swan_evidence = {
    'Political_Instability': 'High',
    'Economic_Weakness': 'High'
}

samples = bn.sample(evidence=black_swan_evidence, n_samples=10000)

# Analyze results
print("Monte Carlo Simulation Results (10,000 runs)")
print("=" * 70)

for node in ['Eurozone_Breakup', 'Corporate_Bonds', 'Government_Bonds', 'Equities']:
    if node in black_swan_evidence:
        continue
    
    print(f"\n{node}:")
    states = bn.states[node]
    for state in states:
        count = samples[node].count(state)
        prob = count / 10000
        print(f"  {state}: {prob:.1%} ({count:,} occurrences)")

## Step 5: Visualize Asset Class Behavior

Compare how different asset classes behave across scenarios.

In [ ]:
# Define scenarios for comparison
scenarios = {
    'Normal': {'Political_Instability': 'Low', 'Economic_Weakness': 'Low'},
    'Political\nCrisis': {'Political_Instability': 'High', 'Economic_Weakness': 'Low'},
    'Economic\nCrisis': {'Political_Instability': 'Low', 'Economic_Weakness': 'High'},
    'Black\nSwan': {'Political_Instability': 'High', 'Economic_Weakness': 'High'}
}

# Calculate probabilities for each scenario
asset_classes = ['Corporate_Bonds', 'Government_Bonds', 'Equities']
adverse_states = {'Corporate_Bonds': 'Falling', 'Government_Bonds': 'Rally', 'Equities': 'Falling'}

results_matrix = np.zeros((len(scenarios), len(asset_classes)))

for i, (scenario_name, evidence) in enumerate(scenarios.items()):
    result = bn.get_probability(evidence)
    for j, asset in enumerate(asset_classes):
        results_matrix[i, j] = result[asset][adverse_states[asset]]

# Plot
fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(scenarios))
width = 0.25

colors = ['#FF6B6B', '#4ECDC4', '#45B7D1']
labels = ['Corporate Bonds (Falling)', 'Government Bonds (Rally)', 'Equities (Falling)']

for j, (asset, label, color) in enumerate(zip(asset_classes, labels, colors)):
    ax.bar(x + j*width, results_matrix[:, j], width, label=label, color=color, alpha=0.8)

ax.set_xlabel('Scenario', fontsize=12, fontweight='bold')
ax.set_ylabel('Probability', fontsize=12, fontweight='bold')
ax.set_title('Asset Class Behavior Under Different Stress Scenarios', fontsize=14, fontweight='bold')
ax.set_xticks(x + width)
ax.set_xticklabels(scenarios.keys())
ax.legend()
ax.grid(axis='y', alpha=0.3)
ax.set_ylim(0, 1)

# Add percentage labels
for i in range(len(scenarios)):
    for j in range(len(asset_classes)):
        height = results_matrix[i, j]
        ax.text(i + j*width, height + 0.02, f'{height:.0%}', 
               ha='center', va='bottom', fontsize=8)

plt.tight_layout()
plt.show()

## Step 6: Portfolio Implications

The key insight: **Government bonds benefit during stress, while corporate bonds and equities suffer.**

This demonstrates the Rebonato-Denev approach to portfolio allocation:
- Don't just optimize for "normal times"
- Explicitly model stress scenarios
- Adjust allocations based on scenario probabilities

In [ ]:
# Simple portfolio allocation example
print("Portfolio Allocation Recommendations")
print("=" * 70)
print()
print("Normal Times:")
print("  - Equities: 60%")
print("  - Corporate Bonds: 30%")
print("  - Government Bonds: 10%")
print()
print("High Stress Probability (>50% Eurozone Breakup):")
print("  - Equities: 20%")
print("  - Corporate Bonds: 10%")
print("  - Government Bonds: 70%")
print()
print("Key Principle: Shift to defensive assets as crisis probability increases")

## Key Takeaways

1. **Causal Structure Matters**: Understanding *why* events happen helps predict *how* they'll propagate

2. **Forward-Looking**: Can model events that haven't happened yet (unlike pure historical approaches)

3. **Expert Judgment**: Combines quantitative rigor with qualitative insights

4. **Dynamic Updates**: As new information arrives, can update beliefs through Bayesian inference

5. **Practical**: Directly applicable to portfolio management and risk management decisions

## Further Exploration

Try modifying:
- The conditional probabilities in the network
- Adding new variables (e.g., "Central_Bank_Intervention")
- Creating different causal structures
- Applying this to other "black swan" scenarios (e.g., pandemic, cyber attack)